In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
debug=False
if debug:
  import numpy as np
  from PIL import Image
  def read_image(path):
    return np.asarray(Image.open(path).convert('L'))
  def write_img(image,path):
    img=Image.fromarray(np.array(image),"L")
    img.save(path) 
TEST_DIR="/content/drive/MyDrive/give/"
train_dir="/content/drive/MyDrive/give/lmao/"
DATA_DIR= "/content/drive/MyDrive/lol4"
TEST_IMG = DATA_DIR+"/t10k-images.idx3-ubyte"
TEST_LAB=DATA_DIR+"/t10k-labels.idx1-ubyte"
TRAIN_IMG=DATA_DIR+"/train-images.idx3-ubyte"
TRAIN_LAB=DATA_DIR+"/train-labels.idx1-ubyte"
# TRAIN_IMG

In [3]:
def bytes_to_int(byte_data):
  return int.from_bytes(byte_data,'big')

In [4]:
def readimg(filename,maximg=None):
  images=[]
  with open(filename,'rb') as f:
    _=bytes_to_int(f.read(4))
    n_images=bytes_to_int(f.read(4))
    if maximg:
      n_images=maximg
    n_rows =bytes_to_int(f.read(4))
    n_col =bytes_to_int(f.read(4))
    for image_idx in range(n_images):
      img=[]
      for row_idx in range(n_rows):
        row=[]
        for col_idx in range(n_col):
          pix=f.read(1)
          row.append(pix)
        img.append(row)
      images.append(img)
    return images

In [5]:
def readlab(filename,maxlab=None):
  labels=[]
  with open(filename,'rb') as f:
    _=bytes_to_int(f.read(4))
    n_lables=bytes_to_int(f.read(4))
    if maxlab:
      n_lables=maxlab
    n_rows =bytes_to_int(f.read(4))
    n_col =bytes_to_int(f.read(4))
    for label_idx in range(n_lables):
      label=f.read(1)
      labels.append(label)
  return labels

In [6]:
def most_freq(l):
  return max(l,key=l.count)

In [7]:
def dist(x,y):
  return sum([(bytes_to_int(i)-bytes_to_int(j))**2 for i,j in zip(x,y)])**0.5

In [8]:
def distance_bw_points(img,X_Train):
  return[dist(img,i)for i in X_Train]

In [9]:
def flatten_list(l):
  return [pixel for sublist in l for pixel in sublist]

In [10]:
def extract_features(filename):
  return [flatten_list(sample) for sample in filename]

In [11]:
def knn(X_Train,Y_train,X_Test,Y_test,k=3):
  y_pred=[]
  for s_idx,sample in enumerate(X_Test):
    training_dist=distance_bw_points(sample,X_Train)
    indices=[
             pair[0] 
             for pair in sorted(list(enumerate(training_dist)),key=lambda x:x[1])
             ]
    # print(Y_test[i] for i in indices)
    candidates=[
                Y_train[idx] 
                for idx in indices[:k]
                ]
    # print(f'Point is {bytes_to_int(Y_test[s_idx])} we guessed')
    # for i in candidates:
    #   print(bytes_to_int(i))
    # print(bytes_to_int(Y_test[i]) for i in candidates)

    
    top=most_freq(candidates)
    y_pred.append(top)
  return y_pred

In [12]:
fX_Train=readimg(TRAIN_IMG,)
X_Test=readimg(TEST_IMG,10)
Y_train=readlab(TRAIN_LAB,)
Y_test=readlab(TEST_LAB,10)
# print(len(Y_train))
# print(len(Y_test))
# print(bytes_to_int(Y_test[0]))
X_train=extract_features(X_Train)
X_test=extract_features(X_Test)
# print(len(X_Train[0]))
# print(len(X_Test[0]))
if debug:
  # for idx,test_sample in enumerate(X_Train):
  #   write_img(test_sample,f'{train_dir}{idx}.png')
  # for i in Y_train:
  #   print("l",bytes_to_int(i))
  for idx,test_sample in enumerate(X_Test):
    write_img(test_sample,f'{TEST_DIR}{idx}.png')
  for i in Y_test:
    print(bytes_to_int(i))
y_pred=knn(X_train,Y_train,X_test,Y_test,)
# print(len(Y_test))
accuracy = sum([
        int(y_pred_i == y_test_i)
        for y_pred_i, y_test_i
        in zip(y_pred, Y_test)
    ]) / len(Y_test)
# for i in y_pred:
#   print(" ")
#   print(bytes_to_int(i))

print(f'Accuracy: {accuracy * 1000}%')
pass


NameError: ignored